In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import io
import json
import re

from gitlab import Gitlab, GitlabListError
from zipfile import ZipFile
from subprocess import run

from tf.advanced.repo import checkoutRepo

In [3]:
GL = "GL"
HC = "HC"

VERSION = "0.2"
REPO = "banks"
COMMIT = "5df379a68800729cc207cde836cd3ba77a0ed018"
MOD = "sim/tf"

ZIPPATH = f"tf%2F{VERSION}"

BACKEND = {GL: "https://gitlab.com", HC: "https://gitlab.huc.knaw.nl"}
ORG = {GL: "annotation-huc", HC: "annotation"}
FREPO = {GL: f"{ORG[GL]}%2F{REPO}", HC: f"{ORG[HC]}%2F{REPO}"}

PERS = {
    GL: os.environ["GL_GITLAB_COM_PERS"],
    HC: os.environ["GL_GITLAB_HUC_KNAW_NL_PERS"],
}
SLUG = {b: f"{bUrl}/api/v4" for (b, bUrl) in BACKEND.items()}
AUTH = {b: f"private_token={token}" for (b, token) in PERS.items()}
PATH = {b: f"{slug}/projects/{FREPO[b]}" for (b, slug) in SLUG.items()}
DOWNLOAD = {
    b: os.path.expanduser(f"~/Downloads/{b}-{REPO}-tf-{VERSION}.zip") for b in BACKEND
}

# Direct via curl

In [4]:
def curl(request, key=None):
    completed = run(f"curl -s {request}", capture_output=True, shell=True)
    response = json.loads(completed.stdout.decode("utf8"))
    if key is not None:
        response = response.get(key, "NOT FOUND")
    if completed.returncode:
        print(completed.stderr)
        return None
    return response

In [5]:
props = dict(GL={"org": ORG[GL], "conn": None}, HC={"org": ORG[HC], "conn": None})

In [6]:
VAR_RE = re.compile(r"""<([^>]+)>""")


def fire(command, key=None, store=None):
    if store is None:
        store = command if key is None else key

    for backend in (GL, HC):

        def varRepl(match):
            var = match.group(1)
            val = str(props[backend].get(var, f"!{var}!"))
            return val

        bCommand = VAR_RE.sub(varRepl, command)
        sep = "&" if "?" in bCommand else "?"
        response = curl(f"{SLUG[backend]}/{bCommand}{sep}{AUTH[backend]}", key=key)
        responseRep = str(response)
        trail = "" if len(responseRep) <= 80 else f" ... {len(responseRep)} chars"
        responseRep = f"{responseRep[0:80]}{trail}"

        if key is not None:
            print(f"<{backend}> {key}={responseRep}")
        else:
            print(f"<{backend}> {responseRep}")
        props[backend][store] = response


def connect():
    for (backend, bUrl) in BACKEND.items():
        props[backend]["conn"] = Gitlab(bUrl, private_token=PERS[backend])


def version():
    for backend in BACKEND:
        conn = props[backend]["conn"]
        response = conn.version()
        print(f"<{backend}> version={response[0]}")
        props[backend]["version"] = response


def project():
    for backend in BACKEND:
        conn = props[backend]["conn"]
        response = conn.projects.get(f"{ORG[backend]}/{REPO}")
        projectId = getattr(response, "id")
        print(f"<{backend}> projectId={projectId}")
        props[backend]["project"] = response
        props[backend]["projectId"] = projectId


def subfolder():
    for backend in BACKEND:
        conn = props[backend]["conn"]
        proj = props[backend]["project"]
        projId = getattr(proj, "id")
        response = conn.http_get(
            f"/projects/{projId}/repository/archive.zip",
            query_data=dict(sha=COMMIT, path=MOD),
            raw=True,
        )
        zf = response.content
        
        with open(DOWNLOAD[backend], "wb") as fh:
            fh.write(zf)

# Connection and version

In [7]:
fire("version", key="version")

<GL> version=15.1.0-pre
<HC> version=12.5.4-ee


# Current user

In [8]:
fire("user", key="id", store="userId")

<GL> id=11713030
<HC> id=52


# Via Python-Gitlab

In [9]:
connect()

In [10]:
version()

<GL> version=15.1.0-pre
<HC> version=12.5.4-ee


In [11]:
project()

<GL> projectId=37114984
<HC> projectId=214


# Subfolder download

Works with gitlab.com, not with gitlab.huc.knaw.nl.

The gitlab version should be at least 14.4.0

In [12]:
subfolder()

## Release creation

In [35]:
if not True:
    project.releases.create(
        dict(
            name="First conversion to TF",
            tag_name="v1.0",
            description="""Features have been produced.
Features have the version as metadata in them.""",
        )
    )

In [36]:
if not True:
    project.releases.create(
        dict(
            name="Version 0.2 of the data",
            tag_name="v2.0",
            description="""Data upgraded to version 0.2.
Readme updated.""",
        )
    )

In [37]:
if not True:
    project.releases.create(
        dict(
            name="Zenodo",
            tag_name="v3.0",
            description="""readme badges""",
        )
    )

In [24]:
if not True:
    project.releases.create(
        dict(
            name="Reorganized",
            tag_name="v3.1",
            description="""There is now an app inside""",
        )
    )